In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

In [111]:
resigned = pd.read_csv("resigned.csv")
reasons = pd.read_csv("reasons.csv")

In [112]:
resigned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 888 entries, 0 to 887
Data columns (total 6 columns):
Date of Leaving    888 non-null object
Staff ID           888 non-null int64
Name of APO        888 non-null object
Org Unit           888 non-null object
Leaving Reason     888 non-null object
Employment         888 non-null object
dtypes: int64(1), object(5)
memory usage: 41.7+ KB


In [113]:
reasons.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1293 entries, 0 to 1292
Data columns (total 29 columns):
Timestamp                                1293 non-null object
Staff ID                                 1290 non-null float64
Cannot adapt to working environement     128 non-null object
Change of career line                    617 non-null object
Difficulty with location                 62 non-null object
Expectations mismatched                  2 non-null object
Family reasons                           151 non-null object
Further studies                          284 non-null object
Health issues unrelated to work          154 non-null object
Health issues related to work            0 non-null float64
Difficulty with job scope                55 non-null object
Lack of career development               12 non-null object
Long working hours                       338 non-null object
Low basic pay                            406 non-null object
No promotional prospects                 9

Drop the duplicate Staff IDs:

In [114]:
resigned.drop_duplicates(subset='Staff ID', keep='first', inplace=True)
reasons.drop_duplicates(subset='Staff ID', keep='first', inplace=True)

Data cleaning:

In [115]:
#Resigned data:
resigned['Date of Leaving'] = pd.to_datetime(resigned['Date of Leaving'])
resigned['Staff ID'] = resigned['Staff ID'].astype(int)
resigned.drop(['Name of APO', 'Leaving Reason', 'Employment'], axis=1, inplace=True)
resigned = resigned[resigned['Org Unit']!='APF HQ Recruits Branch']

In [116]:
#Reasons data:
reasons.drop(['Timestamp'], axis=1, inplace=True)
reasons.replace("Yes", 1, inplace=True)
reasons['Difficulty with location/transport'] = reasons[["Difficulty with location", "Transport problems"]].max(axis=1)
reasons['Cannot adapt to working environment/working conditions'] = reasons[["Cannot adapt to working environement", "Working conditions"]].max(axis=1)
reasons['Health issues'] = reasons[["Health issues unrelated to work", "Health issues related to work"]].max(axis=1)
reasons['Lack of career development/promotion prospects'] = reasons[["Lack of career development", "No promotional prospects"]].max(axis=1)
reasons.drop(['Difficulty with location', 'Transport problems'], axis=1, inplace=True)
reasons.drop(['Cannot adapt to working environement', 'Working conditions'], axis=1, inplace=True)
reasons.drop(['Health issues unrelated to work', 'Health issues related to work'], axis=1, inplace=True)
reasons.drop(['Lack of career development', 'No promotional prospects'], axis=1, inplace=True)
reasons.rename(columns={'Difficulty with clients': 'Difficulties with clients', 'Difficulty with colleagues': 
                        'Difficulties with colleagues', 'Difficulty with supervisors': 'Difficulties with supervisors'}, inplace=True)
reasons['Staff ID'].fillna(0, inplace=True)
reasons['Staff ID'] = reasons['Staff ID'].astype(int)
reasons_melted = pd.melt(reasons, id_vars=['Staff ID'])
reasons_melted.dropna(inplace=True)

In [125]:
reasons.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1278 entries, 0 to 1292
Data columns (total 24 columns):
Staff ID                                                  1278 non-null int64
Change of career line                                     609 non-null float64
Expectations mismatched                                   2 non-null float64
Family reasons                                            150 non-null float64
Further studies                                           283 non-null float64
Difficulty with job scope                                 55 non-null float64
Long working hours                                        337 non-null float64
Low basic pay                                             402 non-null float64
Personal reasons                                          69 non-null float64
Received a better job offer                               211 non-null float64
Left for higher salary                                    7 non-null float64
Retirement/taking a break      

In [124]:
reasons_melted.head()

,Staff ID,variable,value
0,24650,Change of career line,1
1,23995,Change of career line,1
2,36507,Change of career line,1
3,64224,Change of career line,1
4,61357,Change of career line,1


Merge and write:

In [117]:
df2 = pd.merge(reasons_melted, resigned, how='left', on=['Staff ID'])
df2.dropna(inplace=True)
df2['Date of Leaving'] = df2['Date of Leaving'].map(lambda x : x.replace(day=1).date())
df2.to_csv("resigned_clean.csv",index=False)

Headcount and resignations:

In [134]:
df3 = pd.merge(resigned, reasons, how='left', on=['Staff ID'])
df3['Date of Leaving'] = df3['Date of Leaving'].map(lambda x : x.replace(day=1).date())

In [135]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 788 entries, 0 to 787
Data columns (total 26 columns):
Date of Leaving                                           788 non-null object
Staff ID                                                  788 non-null int64
Org Unit                                                  788 non-null object
Change of career line                                     401 non-null float64
Expectations mismatched                                   1 non-null float64
Family reasons                                            81 non-null float64
Further studies                                           147 non-null float64
Difficulty with job scope                                 11 non-null float64
Long working hours                                        228 non-null float64
Low basic pay                                             286 non-null float64
Personal reasons                                          34 non-null float64
Received a better job offer        

In [136]:
df4 = pd.read_csv("headcount.csv")
df5 = df3.groupby(['Org Unit', 'Date of Leaving']).count()['Staff ID'].reset_index()

In [137]:
df4.head()

,Org. Unit,Date,Headcount
0,1B6-AGC,2017-04-01,4
1,1B6-AGC,2017-05-01,4
2,1B6-AGC,2017-06-01,4
3,1B6-AGC,2017-07-01,3
4,1B6-AGC,2017-08-01,3


Merge and write:

In [138]:
df4['Date'] = pd.to_datetime(df4['Date'])
df4['Date'] = df4['Date'] + pd.DateOffset(months=1)
df5.rename(columns={'Org Unit': 'Org. Unit', 'Date of Leaving': 'Date', 'Staff ID': 'Resignations'}, inplace=True)
df5['Date'] = pd.to_datetime(df5['Date'])
df6 = pd.merge(df4, df5, how="left", on=['Org. Unit', 'Date'])
df6.fillna(0, inplace=True)
df6.to_csv("attritions.csv",index=False)